### What:

Trying to get node classification with deepwalk+Label Propagation to work

### TODO:

- improve documentation/comments
- add train and test split to actually test performance for classiying
    - follow the authors code for the split. See main.py in the 'classifier' folder
        - same splits and hyperparameters
    - add code for LabelPropagation() to perform the task
- compare Perozzis embeddings with karateclub
    - make their shapes compatible

In [1]:
import numpy as np
import networkx as nx
from karateclub import LabelPropagation
from sklearn.neighbors import NearestNeighbors
import embed_utils
from sklearn.model_selection import train_test_split
from copy import deepcopy
from collections import Counter

In [2]:
import random
import networkx as nx
from typing import Dict
from karateclub.estimator import Estimator


class MyLabelPropagation(Estimator):
    r"""An implementation of `"Label Propagation Clustering" <https://arxiv.org/abs/0709.2938>`_
    from the Physical Review '07 paper "Near Linear Time Algorithm to Detect Community Structures
    in Large-Scale Networks". The tool executes a series of label propagations with unique labels.
    The final labels are used as cluster memberships.
    Args:
        seed (int): Random seed. Default is 42.
        iterations (int): Propagation iterations. Default is 100.
    """

    def __init__(self, seed: int = 42, iterations: int = 7):
        self.seed = seed
        self.iterations = iterations

    def _make_a_pick(self, neighbors):
        """
        Choosing a neighbor from a propagation source node.
        Arg types:
            * **neigbours** *(list)* - Neighbouring nodes.
        """
        scores = {}
        for neighbor in neighbors:
            neighbor_label = self._labels[neighbor]

            if neighbor_label in scores.keys():
                scores[neighbor_label] = scores[neighbor_label] + 1
            else:
                scores[neighbor_label] = 1

        top = [key for key, val in scores.items() if val == max(scores.values())]
        return random.sample(top, 1)[0]

    def _do_a_propagation(self):
        """
        Doing a propagation round.
        """
        random.shuffle(self._nodes)
        new_labels = {}
        for node in self._nodes:
            neighbors = [neb for neb in nx.neighbors(self._graph, node)]
            pick = self._make_a_pick(neighbors)
            new_labels[node] = pick
        self._labels = new_labels

    def fit(self, graph: nx.classes.graph.Graph):
        """
        Fitting a Label Propagation clustering model.
        Arg types:
            * **graph** *(NetworkX graph)* - The graph to be clustered.
        """
        self._set_seed()
        graph = self._check_graph(graph)
        self._graph = graph
        self._nodes = [node for node in self._graph.nodes()]
        self._labels = nx.get_node_attributes(self._graph, "class") # Hier
        
        random.seed(self.seed)
        for _ in range(self.iterations):
            self._do_a_propagation()

    def get_memberships(self) -> Dict[int, int]:
        r"""Getting the cluster membership of nodes.
        Return types:
            * **memberships** *(dict)* - Node cluster memberships.
        """
        memberships = self._labels
        return memberships

In [5]:
def make_k_nearest_graph(embed, original_graph, k=10):
    """"
    Construct a graph from an embedding using the k-nearest graph algorithm
    embed: embedding
    original_graph: needed for preserving node attributes
    k: nearest k neighbors to a point in the embedding space are used fix edges in the returned graph
    """
    # create new graph based on original nodes and attributes
    k_nearest_graph = nx.Graph()
    k_nearest_graph.add_nodes_from(original_graph)
    nx.set_node_attributes(k_nearest_graph, nx.get_node_attributes(original_graph, "class"),  "class")
    # find the k nearest neighbors for each node and add between the node and these neighbors
    # edges in the new graph 
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(embed)
    _, indices = nbrs.kneighbors(embed)
    k_nearest_links = [[i[0], j] for i in indices for j in i[1:]]
    k_nearest_graph.add_edges_from(k_nearest_links)
    return k_nearest_graph

In [8]:
def get_accuracy(target_nodes, predicted_attributes, target_attributes):
    correct = 0
    total = len(target_nodes)
    for node in target_nodes:
        t = predicted_attributes[node]
        y = target_attributes[node]

        if y == t and t is not None:
            correct += 1

    return correct/total * 100

def get_metrics(test_nodes, original_attributes, predicted_attributes):

    nodes_c0 = [node for node in test_nodes if original_attributes[node] == 0]
    acc_c0 = get_accuracy(nodes_c0, predicted_attributes, original_attributes)

    nodes_c1 = [node for node in test_nodes if original_attributes[node] == 1]
    acc_c1 = get_accuracy(nodes_c1, predicted_attributes, original_attributes)

    acc = get_accuracy(test_nodes, predicted_attributes, original_attributes)
    disparity = np.var([acc_c0, acc_c1])

    metrics = {
        "accuracy":acc,
        "disparity":disparity,
        "accuracy_c0":acc_c0,
        "accuracy_c1":acc_c1,
    }

    return metrics


def run_classification(dataset, method):

    # get graph from data
    graph = embed_utils.data2graph(dataset)
    # print(f"#nodes: {len(graph.nodes())}, #edges (bidirectional): {len(graph.edges())}")

    # Split into train test
    train_nodes, test_nodes = train_test_split(list(graph.nodes()), train_size=.5)
    
    # Set labels of test nodes to other classes
    original_attributes = deepcopy(nx.get_node_attributes(graph, "class"))
    node_to_attribute = nx.get_node_attributes(graph, "class")

    i = max(node_to_attribute.values())
    change_to = {} # Node, new attribute
    for node in graph.nodes():

        if node in test_nodes:
            change_to[node] = i
            i += 1
        else:
            change_to[node] = node_to_attribute[node]

    nx.set_node_attributes(graph, change_to, "class")

    # get embedding from graph using DeepWalk

    embedded_graph = embed_utils.graph2embed(graph, method=method)

    # embed_synth2_karateclub = embed_utils.load_embed("rice", "NC", method="fairwalk", implementation="singer")
    
    # construct new graph from the embedding using k-nearest graph algorithm
    k_nearest_graph = make_k_nearest_graph(embedded_graph, graph)

    # use label propagation on the new graph for classification
    lp = MyLabelPropagation(seed=None, iterations=7)
    lp.fit(k_nearest_graph)

    # Metrics
    metrics = get_metrics(test_nodes, original_attributes, lp.get_memberships())

    return metrics

run_classification(dataset="synth2", method="fairwalk")

UnboundLocalError: local variable 'embed' referenced before assignment

In [145]:
def get_accuracy(target_nodes, predicted_attributes, target_attributes):
    correct = 0
    total = len(target_nodes)
    for node in target_nodes:
        t = predicted_attributes[node]
        y = target_attributes[node]

        if y == t and t is not None:
            correct += 1

    return correct/total

nodes_c0 = [node for node in test_nodes if original_attributes[node] == 0]
acc_c0 = get_accuracy(nodes_c0, lp.get_memberships(), original_attributes)

nodes_c1 = [node for node in test_nodes if original_attributes[node] == 1]
acc_c1 = get_accuracy(nodes_c1, lp.get_memberships(), original_attributes)

print(f"Test nodes in c=0: {len(nodes_c0)}, Test nodes in c=1: {len(nodes_c1)}")
print("Number of nodes predicted in c0 and c1:", len([n for n in lp.get_memberships()if n == 0]), len([n for n in lp.get_memberships() if n == 1]))
print("Most common labels in predictions:", Counter(list(lp.get_memberships().values())).most_common(3))
print(acc_c0*100, acc_c1*100)
np.var([acc_c0*100, acc_c1*100])

Test nodes in c=0: 80, Test nodes in c=1: 170
Number of nodes predicted in c0 and c1: 1 1
Most common labels in predictions: [(1, 445), (0, 26), (249, 1)]
0.0 100.0


2500.0